In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from copy import deepcopy

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import os



In [3]:
## hyperparameters:
num_epochs = 5
batch_size = 100
learning_rate = 0.001

### Load dataset: MNIST

In [4]:
train_dataset  = torchvision.datasets.MNIST(root='./data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                          train=False,
                          transform=transforms.ToTensor())

train_loader  = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/home/neofelia/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
x,y = next(iter(train_loader))
x.shape

torch.Size([100, 1, 28, 28])

## Model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2), #1 * 28 * 28 -> 16 * 28 * 28
            nn.BatchNorm2d(16), #16 * 28 * 28
            nn.ReLU(),                 #16 * 28 * 28
            nn.MaxPool2d(2))       # 16 * 14 * 14
        self.layer2  = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2), # 16 * 14 * 14 -> 32 * 14 * 14
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))       # 32 * 7 * 7
        self.fc = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
model = CNN()

## Train the model

In [ ]:
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
            
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
            
        if (i+1) % 100 == 0:
            print("epoch, ",epoch,"loss - ", loss, )

In [ ]:
print("Model's state_dict:")

for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print()

In [ ]:
for param_tensor in model.state_dict():
    print("\n\n\n",  model.state_dict()[param_tensor])


## Test the model

In [ ]:
         
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
       # images = images.to(device)
       # labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
PATH = './model.pth'
torch.save(model.state_dict(), PATH)